# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [19]:
geoapify_key

'410f4b9802de473293921cce137cf237'

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)
    
# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
fav_temp = city_data_df.loc[(city_data_df["Max Temp"] < 29 ) & (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 4)]

# Drop any rows with null values
fav_temp.dropna()

# Display sample data
fav_temp.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
81,81,makakilo city,21.3469,-158.0858,21.66,81,0,2.57,US,1666108282
90,90,saint george,37.1041,-113.5841,13.33,44,0,1.54,US,1666108286
96,96,zhangye,38.9342,100.4517,11.11,16,0,1.67,CN,1666108292
115,115,cedar city,37.6775,-113.0619,14.11,49,0,0.00,US,1666108304
121,121,lichuan,30.3000,108.8500,9.72,77,0,1.10,CN,1666108308
128,128,half moon bay,37.4636,-122.4286,19.71,86,0,0.00,US,1666108312
141,141,brenham,30.1669,-96.3977,14.47,40,0,3.09,US,1666108319


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = fav_temp.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5,"Hotel Name"," ")

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,
51,hilo,US,19.7297,-155.0900,83,
63,banda,IN,25.4833,80.3333,52,
81,makakilo city,US,21.3469,-158.0858,81,
90,saint george,US,37.1041,-113.5841,44,
96,zhangye,CN,38.9342,100.4517,16,
115,cedar city,US,37.6775,-113.0619,49,
121,lichuan,CN,30.3000,108.8500,77,
128,half moon bay,US,37.4636,-122.4286,86,
141,brenham,US,30.1669,-96.3977,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "filter":radius,
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = bias = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
cedar city - nearest hotel: Baymont Inn & Suites
lichuan - nearest hotel: No hotel found
half moon bay - nearest hotel: Half Moon Bay Inn
brenham - nearest hotel: Ant Street Inn
kahului - nearest hotel: Maui Seaside Hotel
geraldton - nearest hotel: Ocean Centre Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
chengde - nearest hotel: 紫御国际假日酒店
kamen-rybolov - nearest hotel: No hotel found
swan river - nearest hotel: Swan River Super 8
santa rosalia - nearest hotel: Sol y Mar
narwar - nearest hotel: No hotel found
saint anthony - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
90,saint george,US,37.1041,-113.5841,44,"The Advenire, an Autograph Collection Hotel"
96,zhangye,CN,38.9342,100.4517,16,Zhangye Hotel 张掖饭店
115,cedar city,US,37.6775,-113.0619,49,Baymont Inn & Suites
121,lichuan,CN,30.3000,108.8500,77,No hotel found
128,half moon bay,US,37.4636,-122.4286,86,Half Moon Bay Inn
141,brenham,US,30.1669,-96.3977,40,Ant Street Inn


In [30]:
name_address

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': '客房',
    'country': 'China',
    'country_code': 'cn',
    'state': 'Hebei',
    'city': 'Qinhuangdao City',
    'district': 'Changli County',
    'street': '晾甲桥',
    'lon': 119.1758671,
    'lat': 39.7368354,
    'formatted': '客房, China, Hebei, Qinhuangdao City, 晾甲桥',
    'address_line1': '客房',
    'address_line2': 'China, Hebei, Qinhuangdao City, 晾甲桥',
    'categories': ['accommodation', 'accommodation.hotel'],
    'details': [],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database Licence',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': '客房',
      'osm_id': 8461625835,
      'tourism': 'hotel',
      'osm_type': 'n'}},
    'distance': 4222,
    'place_id': '5150bc146841cb5d40598b43559f50de4340f00103f901eb295af801000000920306e5aea2e688bf'},
   'geometry': {'type': 'Point',
    'coordin

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [48]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name","Country"]
)

# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)